## Inputs
---

In [ ]:
# reading data and working with arrays
import os
import zarr, nrrd
import numpy as np
import os

# data paths
p = '/Users/goinac/Work/HHMI/bioimagetools/nextflow-modules/mylocal/bigstream-testdata3'
fix_path = p + '/fix.n5'
ref_ch = 'c0'
fix_s0_subpath = f'{ref_ch}/s0'
mov_path = p + '/mov.n5'
mov_s0_subpath = f'{ref_ch}/s0'
exp_factor = 1

out_p = '/Users/goinac/Work/HHMI/bioimagetools/nextflow-modules/mylocal/results/bigstream-python'
os.makedirs(out_p, exist_ok=True)

# load fix data and spacing
fix_zarr = zarr.open(store=zarr.N5Store(fix_path), mode='r')
fix_meta = fix_zarr[fix_s0_subpath].attrs.asdict()
print(f'Fix attrs: {fix_meta}')

# load mov data and spacing
mov_zarr = zarr.open(store=zarr.N5Store(mov_path), mode='r')
mov_meta = mov_zarr[mov_s0_subpath].attrs.asdict()
print(f'Mov attrs: {mov_meta}')


In [ ]:
!mkdir -p /Users/goinac/Work/HHMI/bioimagetools/nextflow-modules/mylocal/results/bigstream-test

!python ../scripts/main_global_align_pipeline.py \
    --global-fix /Users/goinac/Work/HHMI/bioimagetools/nextflow-modules/mylocal/bigstream-testdata3/fix.n5 \
    --global-fix-subpath c0/s0 \
    --global-mov /Users/goinac/Work/HHMI/bioimagetools/nextflow-modules/mylocal/bigstream-testdata3/mov.n5 \
    --global-mov-subpath c0/s0 \
    --global-registration-steps ransac,affine \
    --align-config /Users/goinac/Work/HHMI/bioimagetools/nextflow-modules/tests/subworkflows/janelia/bigstream_registration/bigstream_config.yml \
    --global-transform-dir /Users/goinac/Work/HHMI/bioimagetools/nextflow-modules/mylocal/results/bigstream-test \
    --global-transform-name affine.mat \
    --global-align-dir /Users/goinac/Work/HHMI/bioimagetools/nextflow-modules/mylocal/results/bigstream-test \
    --global-align-name affine.n5  \
    --global-align-blocksize 128,128,128 \
    --cpus 1 \
    --verbose

In [ ]:

!python ../scripts/main_local_align_pipeline.py \
    --local-fix /Users/goinac/Work/HHMI/bioimagetools/nextflow-modules/mylocal/bigstream-testdata3/fix.n5 \
    --local-fix-subpath c0/s0 \
    --local-mov /Users/goinac/Work/HHMI/bioimagetools/nextflow-modules/mylocal/bigstream-testdata3/mov.n5 \
    --local-mov-subpath c0/s0 \
    --global-affine-transform /Users/goinac/Work/HHMI/bioimagetools/nextflow-modules/mylocal/results/bigstream-test/affine.mat \
    --align-config /Users/goinac/Work/HHMI/bioimagetools/nextflow-modules/tests/subworkflows/janelia/bigstream_registration/bigstream_config.yml \
    --dask-config /Users/goinac/Work/HHMI/bioimagetools/nextflow-modules/tests/subworkflows/janelia/bigstream_registration/dask_config.yml \
    --local-transform-dir /Users/goinac/Work/HHMI/bioimagetools/nextflow-modules/mylocal/results/bigstream-test/local \
    --local-transform-name deform.n5 \
    --local-inv-transform-name inv-deform.n5 \
    --local-align-dir /Users/goinac/Work/HHMI/bioimagetools/nextflow-modules/mylocal/results/bigstream-test/local \
    --local-align-name warped.n5  \
    --local-transform-blocksize 128,128,128 \
    --local-align-blocksize 64,64,64 \
    --worker-cpus 1 \
    --verbose


In [ ]:
!python ../scripts/main_compute_local_inverse.py \
    --transform-dir /Users/goinac/Work/HHMI/bioimagetools/nextflow-modules/mylocal/results/bigstream-test/local \
    --transform-name deform.n5 \
    --transform-subpath c0/s0 \
    --inv-transform-blocksize 128,128,64 \
    --verbose

    

In [ ]:
!mkdir -p /Users/goinac/Work/HHMI/bioimagetools/nextflow-modules/mylocal/results/bigstream-invtest

!python ../scripts/main_compute_local_inverse.py \
    --transform-dir /nrs/scicompsoft/goinac/multifish/tiny/results/local-registration/t1-t2 \
    --transform-name transform \
    --transform-subpath t2/c1/s2 \
    --inv-transform-dir /Users/goinac/Work/HHMI/bioimagetools/nextflow-modules/mylocal/results/bigstream-invtest \
    --verbose


In [ ]:

fix_spacing_s0 = np.array(fix_meta['pixelResolution']['dimensions'][::-1]) / exp_factor
fix_spacing_s1 = fix_spacing_s0 * [2, 2, 2]
fix_spacing_s2 = fix_spacing_s0 * [4, 4, 4]
fix_spacing_s3 = fix_spacing_s0 * [8, 8, 8]
fix_spacing_s4 = fix_spacing_s0 * [16, 16, 16]
fix_spacing_s5 = fix_spacing_s0 * [32, 32, 32]
print(f'fix_spacing_s0: {fix_spacing_s0}')
print(f'fix_spacing_s1: {fix_spacing_s1}')
print(f'fix_spacing_s2: {fix_spacing_s2}')
print(f'fix_spacing_s3: {fix_spacing_s3}')
print(f'fix_spacing_s4: {fix_spacing_s4}')
print(f'fix_spacing_s5: {fix_spacing_s5}')

mov_spacing_s0 = np.array(mov_meta['pixelResolution']['dimensions'][::-1]) / exp_factor
mov_spacing_s1 = mov_spacing_s0 * [2, 2, 2]
mov_spacing_s2 = mov_spacing_s0 * [4, 4, 4]
mov_spacing_s3 = mov_spacing_s0 * [8, 8, 8]
mov_spacing_s4 = mov_spacing_s0 * [16, 16, 16]
mov_spacing_s5 = mov_spacing_s0 * [32, 32, 32]

print(f'mov_spacing_s0: {mov_spacing_s0}')
print(f'mov_spacing_s1: {mov_spacing_s1}')
print(f'mov_spacing_s2: {mov_spacing_s2}')
print(f'mov_spacing_s3: {mov_spacing_s3}')
print(f'mov_spacing_s4: {mov_spacing_s4}')
print(f'mov_spacing_s5: {mov_spacing_s5}')

global_scale = 's0'
fix_affine_spacing = fix_spacing_s0
mov_affine_spacing = mov_spacing_s0

local_scale = 's0'
fix_deform_spacing = fix_spacing_s0
mov_deform_spacing = mov_spacing_s0

In [ ]:
fix_global = fix_zarr[f'/{ref_ch}/{global_scale}'][...]
mov_global = mov_zarr[f'/{ref_ch}/{global_scale}'][...]
print(f'Fix global shape: {fix_global.shape}: {fix_affine_spacing}')
print(f'Mov global shape: {mov_global.shape}: {mov_affine_spacing}')
nrrd.write(out_p+'/fix.nrrd', fix_global.transpose(2,1,0), compression_level=2)
nrrd.write(out_p+'/mov.nrrd', mov_global.transpose(2,1,0), compression_level=2)

## Alignment
---

### Compute Affine

In [ ]:
# alignment functions
from bigstream.align import alignment_pipeline
from bigstream.transform import apply_transform


In [ ]:

# get global alignment channels
fix = fix_global
mov = mov_global

# define alignment steps
affine_kwargs = {
    'initial_condition': 'CENTER',
    'alignment_spacing':4.0, # increase this to throw out data
    'shrink_factors':[2],
    'smooth_sigmas':[2],
    'optimizer_args':{
        'learningRate':0.25,
        'minStep':0.,
        'numberOfIterations':100,
    },
}

steps = [('affine', affine_kwargs)]

# align
affine = alignment_pipeline(
    fix, mov,
    fix_affine_spacing, mov_affine_spacing,
    steps,
)

### Apply affine

In [ ]:
# apply affine only
affine_aligned = apply_transform(
    fix, mov,
    fix_affine_spacing, mov_affine_spacing,
    transform_list=[affine,],
)


In [ ]:
# Save affine
os.makedirs(out_p, exist_ok=True)
np.savetxt(out_p+'/affine.mat', affine)

In [ ]:
# write affine transformed vol
os.makedirs(out_p, exist_ok=True)
nrrd.write(out_p+'/affine.nrrd', affine_aligned.transpose(2,1,0), compression_level=2)

In [ ]:
# load precomputed results
affine = np.loadtxt(out_p+'/affine.mat')
affine

### Prepare cluster args

In [ ]:
cluster_kwargs={
    'config':{
        'distributed.worker.memory.target':0.9,
        'distributed.worker.memory.spill':0.9,
        'distributed.worker.memory.pause':0.9,
        'distributed.comm.retry.count':10,
        'distributed.comm.timeouts.connect':'600s',
        'distributed.scheduler.worker-saturation': 1,
        'distributed.scheduler.unknown-task-duration':'60m',
    },
}

### Calculate deform transformation

In [ ]:
%%time
from bigstream.piecewise_align import distributed_piecewise_alignment_pipeline

# FASTER

# get full resolution data
fix = fix_zarr[f'/{ref_ch}/{local_scale}']
mov = mov_zarr[f'/{ref_ch}/{local_scale}']

# define alignment steps
affine_kwargs = {
    'smooth_sigmas':(0.25,),
    'optimizer_args':{
        'learningRate':0.25,
        'minStep':0.,
        'numberOfIterations':25,
    },
    "alignment_spacing": 1.0,
}

deform_kwargs = {
    'smooth_sigmas':(0.25,),
    'control_point_spacing':50.0,
    'control_point_levels':(1,),
    'optimizer_args':{
        'learningRate':2.5,
        'minStep':0.,
        'numberOfIterations':25,
    },
    "alignment_spacing": 2.0,
}

steps = [ ('affine', affine_kwargs,), ('deform', deform_kwargs,), ]

# deform
deform = distributed_piecewise_alignment_pipeline(
    fix, mov,
    fix_deform_spacing, mov_deform_spacing,
    steps=steps,
    blocksize=[128, 128, 128],
    static_transform_list=[affine,],
    write_path=out_p+'/deform.zarr',
    cluster_kwargs=cluster_kwargs,
)

In [ ]:
# read already saved result
deform = zarr.open(out_p+'/deform.zarr', mode='r')

### Apply deformation

In [ ]:
from bigstream.piecewise_transform import distributed_apply_transform

blocksize = (64,)*3
channels = [ref_ch]

fix = fix_zarr[f'/{ref_ch}/{local_scale}']
for channel in channels:
    mov = mov_zarr[f'/{channel}/{local_scale}']
    aligned = distributed_apply_transform(
        fix, mov,
        fix_deform_spacing, mov_deform_spacing,
        transform_list=[affine, deform,],
        transform_spacing=((1,1,1), fix_deform_spacing),
        blocksize=blocksize,
        cluster_kwargs=cluster_kwargs
    )
    print(f'Write aligned result to:{out_p}/r2{channel}{local_scale}_aligned_to_r1.nrrd')
    nrrd.write(out_p+f'/r2{channel}{local_scale}_aligned_to_r1.nrrd', aligned.transpose(2,1,0), compression_level=2)


Compare deform output for nextflow and jupyter runs

In [ ]:
jupyter_deform_zarr_name='/Users/goinac/Work/HHMI/bioimagetools/nextflow-modules/mylocal/results/bigstream-python/deform.zarr'
nf_deform_n5_name='/Users/goinac/Work/HHMI/bioimagetools/nextflow-modules/mylocal/results/bigstream-test/local/deform.n5'

jupyter_deform_zarr = zarr.open(jupyter_deform_zarr_name, mode='r')
nf_deform_n5_container = zarr.open(store=zarr.N5FSStore(nf_deform_n5_name), mode='r')
nf_deform_n5 = nf_deform_n5_container['c0/s0']


In [ ]:
import zarr
deform_name='/nrs/scicompsoft/goinac/multifish/tiny/results/registration/t3-t5/transform'
deform_dataset='s2'

deform_zarr_container=zarr.open(store=zarr.N5FSStore(deform_name))
deform_zarr = deform_zarr_container[deform_dataset]
deform_arr = deform_zarr[...]

In [ ]:
deform_arr

In [ ]:
print(jupyter_deform_zarr.shape)
jupyter_deform_zarr[0,0:5, 0]

In [ ]:
print(nf_deform_n5.shape)
nf_deform_n5[0,0:5, 0]

In [ ]:
np.abs(jupyter_deform_zarr[...] - nf_deform_n5[...])

In [ ]:
jupyter_deform_zarr[...]-nf_deform_n5
np.testing.assert_allclose(jupyter_deform_zarr[...], nf_deform_n5, rtol=1e-10, atol=1e-06)

In [ ]:

np.all( jupyter_deform_zarr[0:10,0:5,0:1] == nf_deform_n5[0:10,0:5,0:1])

In [ ]:
inv_name = 'local-registration/t1-t2/inv-deform.n5'
inv_ref_name = 'local-registration/t1-t2/inv-test.n5'
dataset = 't2/c1/s0'

inv_zarr_container = zarr.open(store=zarr.N5FSStore(inv_name), mode='r')
inv_ref_zarr_container = zarr.open(store=zarr.N5FSStore(inv_ref_name), mode='r')

inv_zarr = inv_zarr_container[dataset]
inv_ref_zarr = inv_zarr_container[dataset]

inv_arr = inv_zarr[...]
inv_ref_arr = inv_ref_zarr[...]
